In [22]:
import numpy as np
import pandas as pd
import glob
import os
from scipy.stats import pearsonr
import anndata as ad
from scipy.sparse import csr_matrix
from scipy.stats import zscore
import time
from scipy.sparse import issparse
import re

In [23]:
# Bed 
bed = pd.read_csv("/mnt/c/Users/donna/Downloads/Thesis/TSSClassification/beds/dts_beds/atac_match.bed", sep='\t', header=None)

In [24]:
# cfDNA
ff_path = '/mnt/c/Users/donna/Downloads/Thesis/data/cftest'
feature = 'cov_spread'
file_names = [f for f in os.listdir(ff_path) if f.endswith(".csv")]

In [25]:
# atac
atac_df = pd.read_csv("/mnt/c/Users/donna/Downloads/Thesis/tacje/tissue_mean_atac_signals.csv", index_col=0, header=0)

In [ ]:
bed_genes = bed.iloc[:, 3].tolist()
filtered_atac = atac_df.loc[:, atac_df.columns.isin(bed_genes)]

In [26]:
cf_samples = []

# Load each cfDNA file and append to list
for ff_file in file_names:
    cf_file_path = os.path.join(ff_path, ff_file)
    cf_sample = pd.read_csv(cf_file_path, header=0, dtype='object', usecols=[feature])
    cf_sample = cf_sample.astype(float).values.flatten()
    cf_samples.append(cf_sample)
    ## Elke row is een sample

sample_names = [re.match(r"^[^\.]+", f).group(0) for f in file_names]
cf_df = pd.DataFrame(cf_samples, index=sample_names)

In [41]:
cf_z = (cf_df - cf_df.mean(axis=1).values[:, None]) / cf_df.std(axis=1).values[:, None]
atac_z = (filtered_atac - filtered_atac.mean(axis=1).values[:, None]) / filtered_atac.std(axis=1).values[:, None]

# Compute pairwise Pearson correlation (dot product of z-scores)
corr_matrix = np.dot(cf_z.values, atac_z.values.T) / cf_z.shape[1]
corr_matrix = np.abs(corr_matrix)

# Convert to DataFrame with proper labels
corr_df = pd.DataFrame(corr_matrix, index=cf_df.index, columns=atac_df.index)

In [42]:
print(corr_df)

                  adipose_omentum  adrenal_gland  artery_aorta  \
control--EE88162         0.016258       0.020607      0.014448   

                  esophagus_mucosa  heart_lv     liver      lung  \
control--EE88162          0.015769  0.018661  0.019001  0.019661   

                  mammary_tissue    muscle     ovary  pancreas      skin  \
control--EE88162        0.015557  0.016343  0.014433  0.018658  0.019318   

                  small_intestine   stomach   thyroid    uterus    vagina  
control--EE88162         0.018617  0.017241  0.016188  0.014862  0.013547  


In [43]:
ranked_df = corr_df.rank(axis=1, ascending=False, method='min')


In [ ]:
corr_df.to_csv("tacje/abs_cor_mat.csv")
ranked_df.to_csv("tacje/ranked_cor_mat.csv")